## Prompt files
JG: created around 2024-11-18 got it to work on 2024-11-24

## Update prompted ELN with the info form the prompted XLS file
2024-11-24 JG: Yop, that works too

In [2]:
"""# FileBrowser"""

from pathlib import Path
from typing import Optional, cast

import solara
from solara.website.utils import apidoc

import os

from ELNfiller import update_yaml_with_excel_data

#clicks = solara.reactive(0)

@solara.component
def Page():

    #current_dir = os.getcwd()

    file, set_file = solara.use_state(cast(Optional[Path], None))
    path, set_path = solara.use_state(cast(Optional[Path], None))
    #directory, set_directory = solara.use_state(Path("~").expanduser())
    directory, set_directory = solara.use_state(None)

    

    with solara.VBox() as main:
         can_select = solara.ui_checkbox("Enable select")

         def reset_path():
             set_path(None)
             set_file(None)

         #reset path and file when can_select changes
         solara.use_memo(reset_path, [can_select])
         solara.FileBrowser(directory, on_directory_change=set_directory, on_path_select=set_path, on_file_open=set_file, can_select=can_select)
         solara.Info(f"You are in directory: {directory}")
         solara.Info(f"Single click on the ELN: {path}")
         solara.Info(f"Double click on the XLS: {file}")

          #now trying to make something with those files
         def handle_button_click():
            update_yaml_with_excel_data(file, path,
                'Materials',['Usual name','CAS ID','CAS name', 
                'Hill Molecule Formula','CAS synonyms'], 
                ['sample>substance>name','sample>substance>cas_number',
                    'sample>substance>cas_name','sample>substance>molecular_formula_hill','sample>substance>cas_synonyms'])


         solara.Button(label="Update ELN file", 
                        on_click = handle_button_click)
        
        
        # def increase_clicks():
        #     clicks.value += 1

        # solara.Button(label=f"Clicked {clicks} times", on_click=increase_clicks)

    return main


__doc__ += apidoc(solara.FileBrowser.f)  # type: ignore

Page()

Cannot show ipywidgets in text

## Improve interface to create Nexus file

2024-11-25 JG: so basically now I want to add a second column that will ask for the data file and create the nexus file and then upload it to Nomad

In [3]:
from pathlib import Path
import solara
from solara.website.utils import apidoc

# Import your custom module
from ELNfiller import update_yaml_with_excel_data

class SharedState:
    def __init__(self):
        self.file = None
        self.path = None

shared_state = SharedState()

@solara.component
def ELN_updater():
    global shared_state
    
    with solara.VBox() as main:
        can_select = solara.ui_checkbox("Enable select")
        
        def reset_path():
            shared_state.path = None
            shared_state.file = None

        file, set_file = solara.use_state(Path())
        path, set_path = solara.use_state(Path())
        directory, set_directory = solara.use_state(None)

        def handle_file_change(file_path):
            if file_path:
                shared_state.file = Path(file_path)
                shared_state.path = Path(file_path).parent
            else:
                shared_state.file = None
                shared_state.path = None

        solara.FileBrowser(directory, on_directory_change=set_directory, 
                           on_path_select=handle_file_change,
                           on_file_open=handle_file_change,
                           can_select=can_select)
        solara.Info(f"You are in directory: {shared_state.path}")
        solara.Info(f"Single click on the ELN: {shared_state.file}")


    def handle_button_click():
        update_yaml_with_excel_data(shared_state.file, shared_state.path,
            'Materials',['Usual name','CAS ID','CAS name', 
            'Hill Molecule Formula','CAS synonyms'], 
            ['sample>substance>name','sample>substance>cas_number',
                'sample>substance>cas_name','sample>substance>molecular_formula_hill','sample>substance>cas_synonyms'])

    solara.Button(label="Update ELN file", 
                    on_click=handle_button_click)

@solara.component
def Page():
    with solara.Column():
        with solara.Columns([1, 1]):
            ELN_updater()
            ELN_updater()

__doc__ += apidoc(solara.FileBrowser.f)  # type: ignore

Page()


Cannot show ipywidgets in text

25-11-2024 JG: oki. kindof works for creating columns. Just I have to put the ELN updated in the first column too and to make a different thing box for the second column.

Oki a bit better but we lost the interactivity -> to finish later

26-11-2024 JG: Ok, I think the problem is that the way file and path are defined they can't update. Phind only proposes not working solutions. let's revert back to something closer to the initial case.




In [4]:
from pathlib import Path
import solara
from solara.website.utils import apidoc


# Import your custom module
from ELNfiller import update_yaml_with_excel_data

# class SharedState:
#     def __init__(self):
#         self.file = None
#         self.path = None

# shared_state = SharedState()

User_Excel_ELN = solara.reactive(None)
Setup_ELN = solara.reactive(None)
Data_file = solara.reactive(None)


@solara.component
def ELN_updater():
    
    User_Excel_ELN, set_file = solara.use_state(cast(Optional[Path], None))
    Setup_ELN, set_path = solara.use_state(cast(Optional[Path], None))
    directory, set_directory = solara.use_state(None)

    
    with solara.VBox() as main:
         can_select = solara.ui_checkbox("Enable select")

         def reset_path():
             set_path(None)
             set_file(None)

         #reset path and file when can_select changes
         solara.use_memo(reset_path, [can_select])
         solara.FileBrowser(directory, on_directory_change=set_directory, on_path_select=set_path, on_file_open=set_file, can_select=can_select)
         solara.Info(f"You are in directory: {directory}")
         solara.Info(f"Single click on the ELN: {Setup_ELN}")
         solara.Info(f"Double click on the XLS: {User_Excel_ELN}")

          #now trying to make something with those files
         def handle_button_click():
            update_yaml_with_excel_data(User_Excel_ELN, Setup_ELN,
                'Materials',['Usual name','CAS ID','CAS name', 
                'Hill Molecule Formula','CAS synonyms'], 
                ['sample>substance>name','sample>substance>cas_number',
                    'sample>substance>cas_name','sample>substance>molecular_formula_hill','sample>substance>cas_synonyms'])


         solara.Button(label="Update ELN file", 
                        on_click = handle_button_click)

@solara.component
def Page():
    with solara.Column():
        with solara.Columns([1, 1]):
            ELN_updater()
            ELN_updater()

__doc__ += apidoc(solara.FileBrowser.f)  # type: ignore

Page()


Cannot show ipywidgets in text

26-11-2024 JG; oki, works, now change the second one

In [5]:
from pathlib import Path
import solara
from solara.website.utils import apidoc


# Import your custom module
from ELNfiller import update_yaml_with_excel_data
from pynxtools.dataconverter.convert import convert



# class SharedState:
#     def __init__(self):
#         self.file = None
#         self.path = None

# shared_state = SharedState()

User_Excel_ELN = solara.reactive(None)
Setup_ELN = solara.reactive(None)
Data_file = solara.reactive(None)


@solara.component
def ELN_updater():
    
    User_Excel_ELN, set_file = solara.use_state(cast(Optional[Path], None))
    Setup_ELN, set_path = solara.use_state(cast(Optional[Path], None))
    directory, set_directory = solara.use_state(None)

    
    with solara.VBox() as main:
         can_select = solara.ui_checkbox("Enable select")

         def reset_path():
             set_path(None)
             set_file(None)

         #reset path and file when can_select changes
         solara.use_memo(reset_path, [can_select])
         solara.FileBrowser(directory, on_directory_change=set_directory, on_path_select=set_path, on_file_open=set_file, can_select=can_select)
         solara.Info(f"You are in directory: {directory}")
         solara.Info(f"Single click on the ELN: {Setup_ELN}")
         solara.Info(f"Double click on the XLS: {User_Excel_ELN}")

          #now trying to make something with those files
         def handle_button_click():
            update_yaml_with_excel_data(User_Excel_ELN, Setup_ELN,
                'Materials',['Usual name','CAS ID','CAS name', 
                'Hill Molecule Formula','CAS synonyms'], 
                ['sample>substance>name','sample>substance>cas_number',
                    'sample>substance>cas_name','sample>substance>molecular_formula_hill','sample>substance>cas_synonyms'])


         solara.Button(label="Update ELN file", 
                        on_click = handle_button_click)
         
@solara.component
def Nexus_file_and_upload():
    
    Data_file, set_file = solara.use_state(cast(Optional[Path], None))
    Setup_ELN, set_path = solara.use_state(cast(Optional[Path], None))
    directory, set_directory = solara.use_state(None)

    
    with solara.VBox() as main:
         can_select = solara.ui_checkbox("Enable select")

         def reset_path():
             set_path(None)
             set_file(None)

         #reset path and file when can_select changes
         solara.use_memo(reset_path, [can_select])
         solara.FileBrowser(directory, on_directory_change=set_directory, on_path_select=set_path, on_file_open=set_file, can_select=can_select)
         solara.Info(f"You are in directory: {directory}")
         solara.Info(f"Double click on the DataFile: {Data_file}")

          #now trying to make something with those files
         def handle_button_click():
             convert([str(Data_file), str(Setup_ELN)], "xps", "NXmpes", "test_interface_D18.nxs", config_file = r'C:\Users\gorenfjf\Documents\My documents\Data_management\Setups\pynxtools-xps\src\pynxtools_xps\config\config_vms.json')
                    
         solara.Button(label="Generate NXS file", 
                        on_click = handle_button_click)

@solara.component
def Page():
    with solara.Column():
        with solara.Columns([1, 1]):
            ELN_updater()
            Nexus_file_and_upload()

__doc__ += apidoc(solara.FileBrowser.f)  # type: ignore

Page()


Cannot show ipywidgets in text

2024-11-27 JG: Oki looks good, now adding the upload

In [6]:
from pathlib import Path
import solara
from solara.website.utils import apidoc


# Import your custom module
from ELNfiller import update_yaml_with_excel_data
from pynxtools.dataconverter.convert import convert

from Nomad_API import *



# class SharedState:
#     def __init__(self):
#         self.file = None
#         self.path = None

# shared_state = SharedState()

User_Excel_ELN = solara.reactive(None)
Setup_ELN = solara.reactive(None)
Data_file = solara.reactive(None)


@solara.component
def ELN_updater():
    
    User_Excel_ELN, set_file = solara.use_state(None)
    Setup_ELN, set_path = solara.use_state(None)
    directory, set_directory = solara.use_state(None)

    
    with solara.VBox() as main:
         can_select = solara.ui_checkbox("Enable select")

         def reset_path():
             set_path(None)
             set_file(None)

         #reset path and file when can_select changes
         solara.use_memo(reset_path, [can_select])
         solara.FileBrowser(directory, on_directory_change=set_directory, on_path_select=set_path, on_file_open=set_file, can_select=can_select)
         solara.Info(f"You are in directory: {directory}")
         solara.Info(f"Single click on the ELN: {Setup_ELN}")
         solara.Info(f"Double click on the XLS: {User_Excel_ELN}")

          #now trying to make something with those files
         def handle_button_click():
            update_yaml_with_excel_data(User_Excel_ELN, Setup_ELN,
                'Materials',['Usual name','CAS ID','CAS name', 
                'Hill Molecule Formula','CAS synonyms'], 
                ['sample>substance>name','sample>substance>cas_number',
                    'sample>substance>cas_name','sample>substance>molecular_formula_hill','sample>substance>cas_synonyms'])


         solara.Button(label="Update ELN file", 
                        on_click = handle_button_click)
         
@solara.component
def Nexus_file_and_upload():
    
    Data_file, set_file = solara.use_state(None)
    Setup_ELN, set_path = solara.use_state(None)
    directory, set_directory = solara.use_state(None)

    
    with solara.VBox() as main:
         can_select = solara.ui_checkbox("Enable select")

         def reset_path():
             set_path(None)
             set_file(None)

         #reset path and file when can_select changes
         solara.use_memo(reset_path, [can_select])
         solara.FileBrowser(directory, on_directory_change=set_directory, on_path_select=set_path, on_file_open=set_file, can_select=can_select)
         solara.Info(f"You are in directory: {directory}")
         solara.Info(f"Double click on the DataFile: {Data_file}")

          #now trying to make something with those files
         def handle_button_click():
             #TODO: interface to prompt the File name
             convert([str(Data_file), str(Setup_ELN)], "xps", "NXmpes", "test_interface_D18.nxs", config_file = r'C:\Users\gorenfjf\Documents\My documents\Data_management\Setups\pynxtools-xps\src\pynxtools_xps\config\config_vms.json')
                    
         solara.Button(label="Generate NXS file", 
                        on_click = handle_button_click)
         
         def handle_button2_click():
             #TODO: interface to prompt the user later
             username = 'julien.gorenflot@kaust.edu.sa'
             password = 'tIFSMPDL!904'

             nomad_url = 'http://10.72.203.137/nomad-oasis/api/v1/'
             token = get_authentication_token(nomad_url, username, password)
             
             #TODO: prompt dataset and upload ID name
             dataset_id = create_dataset(nomad_url, token, 'Test_Dataset')
             upload_id = upload_to_NOMAD(nomad_url + 'uploads?file_name=test_interface_D18.nxs', token, 'test_interface_D18.nxs')


             last_status_message = check_upload_status(nomad_url, token, upload_id)
             print(last_status_message)

             metadata = {
                "metadata": {
                "upload_name": 'Test_Upload',
                "references": ["https://doi.org/xx.xxxx/x.xxxx"],
                "datasets": dataset_id,
                "embargo_length": 0,
                "coauthor0s": ["jules.bertrandie@kaust.edu.sa@affiliation.de"],
                "comment": 'This is a test upload...',
             },
             }
             response = edit_upload_metadata(nomad_url, token, upload_id, metadata)
            
             last_status_message = check_upload_status(nomad_url, token, upload_id)
             print(last_status_message)
         
             #I put the publishing command lines there, but 'm not sure we should use them yet so I just put them as commands
             # response = publish_upload(nomad_url, token, upload_id)
             # last_status_message = check_upload_status(nomad_url, token, upload_id)
             # print(last_status_message)





             
                    
         solara.Button(label="Upload_To_Nomad_Oasis", 
                        on_click = handle_button2_click)

@solara.component
def Page():
    with solara.Column():
        with solara.Columns([1, 1]):
            ELN_updater()
            Nexus_file_and_upload()

__doc__ += apidoc(solara.FileBrowser.f)  # type: ignore

Page()


ModuleNotFoundError: No module named 'Nomad_API'

2024-12-05 JG: ok. So it works, now need to improve a few things:

    1. pass the file name from the app
    2. change the file pass to drop
    3. check if the dataset already exists
    4. prompt dataset name
    5. keep a copy of the ELN safe and create a new one when adding the sample info
    6. one button less? (ELN)
    7. prompt metadata?
    8. include vernissage


So so first point. Problem is: the file name is not saved anywhere so far -> need to create a varibale with it (+ later maybe we can prompt it too, or create it from something (like the datafile name for example)

In [ ]:
from pathlib import Path
import solara
from solara.website.utils import apidoc


# JG:2025-01-29: force reinstall nomad-API as for some reason it doesn't seem to be found
#!pip install nomad-api --force-reinstall


# Import your custom module

from ELNfiller import update_yaml_with_excel_data
from pynxtools.dataconverter.convert import convert

from Nomad_API import *



# class SharedState:
#     def __init__(self):
#         self.file = None
#         self.path = None

# shared_state = SharedState()

User_Excel_ELN = solara.reactive(None)
Setup_ELN = solara.reactive(None)
Data_file = solara.reactive(None)


@solara.component
def ELN_updater():
    
    User_Excel_ELN, set_file = solara.use_state(None)
    Setup_ELN, set_path = solara.use_state(None)
    directory, set_directory = solara.use_state(None)

    
    with solara.VBox() as main:
         can_select = solara.ui_checkbox("Enable select")

         def reset_path():
             set_path(None)
             set_file(None)

         #reset path and file when can_select changes
         solara.use_memo(reset_path, [can_select])
         solara.FileBrowser(directory, on_directory_change=set_directory, on_path_select=set_path, on_file_open=set_file, can_select=can_select)
         solara.Info(f"You are in directory: {directory}")
         solara.Info(f"Single click on the ELN: {Setup_ELN}")
         solara.Info(f"Double click on the XLS: {User_Excel_ELN}")

          #now trying to make something with those files
         def handle_button_click():
            update_yaml_with_excel_data(User_Excel_ELN, Setup_ELN,
                'Materials',['Usual name','CAS ID','CAS name', 
                'Hill Molecule Formula','CAS synonyms'], 
                ['sample>substance>name','sample>substance>cas_number',
                    'sample>substance>cas_name','sample>substance>molecular_formula_hill','sample>substance>cas_synonyms'])
            update_yaml_with_excel_data(User_Excel_ELN, Setup_ELN,
                'Sample preparation',['Sample preparation date','Sample preparation date','Sample preparation date','Sample preparation date'], 
                ['sample>history>sample_preparation>start_time','sample>history>sample_preparation>end_time','start_time','end_time'])


         solara.Button(label="Update ELN file", 
                        on_click = handle_button_click)
         
@solara.component
def Nexus_file_and_upload():
    
    Data_file, set_file = solara.use_state(None)
    Setup_ELN, set_path = solara.use_state(None)
    directory, set_directory = solara.use_state(None)

    
    with solara.VBox() as main:
         can_select = solara.ui_checkbox("Enable select")

         def reset_path():
             set_path(None)
             set_file(None)

         #reset path and file when can_select changes
         solara.use_memo(reset_path, [can_select])
         solara.FileBrowser(directory, on_directory_change=set_directory, on_path_select=set_path, on_file_open=set_file, can_select=can_select)
         solara.Info(f"You are in directory: {directory}")
         solara.Info(f"Double click on the DataFile: {Data_file}")
         
         Nexus_file_name = str(Data_file)[:-3] + 'nxs'


          #now trying to make something with those files
         def handle_button_click():
             #TODO: interface to prompt the File name 
             #2024-12-05: changed the filename to the datafile name + nxs extension
             convert([str(Data_file), str(Setup_ELN)], "xps", "NXmpes", Nexus_file_name, config_file = r'C:\Users\gorenfjf\Documents\My documents\Data_management\Setups\pynxtools-xps\src\pynxtools_xps\config\config_vms.json')
                    
         solara.Button(label="Generate NXS file", 
                        on_click = handle_button_click)
         
         def handle_button2_click():
             #TODO: interface to prompt the user later
             username = 'julien.gorenflot@kaust.edu.sa'
             password = 'tIFSMPDL!904'

             nomad_url = 'http://10.72.203.137/nomad-oasis/api/v1/'
             token = get_authentication_token(nomad_url, username, password)
             
             #TODO: prompt dataset and upload ID name
             dataset_id = create_dataset(nomad_url, token, 'Test_Dataset')
             upload_id = upload_to_NOMAD(nomad_url + 'uploads?file_name=' + Nexus_file_name, token, Nexus_file_name)


             last_status_message = check_upload_status(nomad_url, token, upload_id)
             print(last_status_message)

             metadata = {
                "metadata": {
                "upload_name": 'Test_Upload',
                "references": ["https://doi.org/xx.xxxx/x.xxxx"],
                "datasets": dataset_id,
                "embargo_length": 0,
                "coauthor0s": ["jules.bertrandie@kaust.edu.sa@affiliation.de"],
                "comment": 'This is a test upload...',
             },
             }
             response = edit_upload_metadata(nomad_url, token, upload_id, metadata)
            
             last_status_message = check_upload_status(nomad_url, token, upload_id)
             print(last_status_message)
         
             #I put the publishing command lines there, but 'm not sure we should use them yet so I just put them as commands
             # response = publish_upload(nomad_url, token, upload_id)
             # last_status_message = check_upload_status(nomad_url, token, upload_id)
             # print(last_status_message)





             
                    
         solara.Button(label="Upload_To_Nomad_Oasis", 
                        on_click = handle_button2_click)

@solara.component
def Page():
    with solara.Column():
        with solara.Columns([1, 1]):
            ELN_updater()
            Nexus_file_and_upload()

__doc__ += apidoc(solara.FileBrowser.f)  # type: ignore

Page()




Cannot show ipywidgets in text

2024-12-05 JG: oki, point 1 managed. But I realize I forgot a very important point: query user ID and pwd 

-> updated todo list:

2024-12-05 JG: ok. So it works, now need to improve a few things:

    1. [DONE] pass the file name from the app
    2. change the file pass to drop
    3. check if the dataset already exists
    4. prompt dataset name
    5. keep a copy of the ELN safe and create a new one when adding the sample info
    6. one button less? (ELN)
    7. prompt user name and pwd
    8. prompt metadata?
    9. include vernissage
    10. enable multpile files at once? (by adding an identifier in the excel file)



In [1]:
import textwrap
from typing import List, cast

import solara
from solara.components.file_drop import FileInfo
from solara.website.utils import apidoc


@solara.component
def FileDropMultipleDemo():
    content, set_content = solara.use_state(cast(List[bytes], []))
    filename, set_filename = solara.use_state(cast(List[str], []))
    size, set_size = solara.use_state(cast(List[int], []))

    def on_file(files: List[FileInfo]):
        set_filename([f["name"] for f in files])
        set_size([f["size"] for f in files])
        set_content([f["file_obj"].read(100) for f in files])

    solara.FileDropMultiple(
        label="Drag and drop files(s) here to read the first 100 bytes.",
        on_file=on_file,
        lazy=True,  # We will only read the first 100 bytes
    )
    if content:
        solara.Info(f"Number of uploaded files: {len(filename)}")
        for f, s, c in zip(filename, size, content):
            solara.Info(f"File {f} has total length: {s}\n, first 100 bytes:")
            solara.Preformatted("\n".join(textwrap.wrap(repr(c))))

@solara.component
def Page2():

    with solara.Card(title="FileDropMultiple"):
        FileDropMultipleDemo()


__doc__ += "# FileDropMultiple"
__doc__ += apidoc(solara.FileDropMultiple.f)  # type: ignore

Page2()


Cannot show ipywidgets in text

In [3]:
!where python
!python -m pip list

c:\Users\gorenfjf\Documents\My documents\Data_management\.conda\python.exe
C:\Users\gorenfjf\.pyenv\pyenv-win\shims\python
C:\Users\gorenfjf\.pyenv\pyenv-win\shims\python.bat
C:\Users\gorenfjf\AppData\Local\Microsoft\WindowsApps\python.exe
C:\Users\gorenfjf\Anaconda3\python.exe
Package                   Version                             Editable project location
------------------------- ----------------------------------- --------------------------------------------------------------------------------------------------------------------------------------
aiohappyeyeballs          2.4.3
aiohttp                   3.11.7
aiosignal                 1.3.1
aniso8601                 9.0.1
anyio                     4.4.0
anytree                   2.12.1
appdirs                   1.4.4
argcomplete               3.5.0
ase                       3.22.1
astroid                   3.2.4
asttokens                 2.4.1
async-property            0.2.2
attrs                     24.2.0
beartype        

In [7]:
import sys
print(sys.executable)

c:\Users\gorenfjf\Documents\My documents\Data_management\.conda\python.exe


In [2]:
!python -m pip freeze > requirements.txt